In [1]:
import httpx
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
# async with httpx.AsyncClient() as client:
#     overall_data = await client.get('https://fantasy.premierleague.com/api/bootstrap-static/')
overall_data = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
overall_data = overall_data.json()

In [3]:
df = pd.DataFrame(overall_data['elements'])

In [4]:
df.index=df['id']

In [5]:
columns_to_remove = ['first_name','photo','news','news_added','second_name','squad_number','web_name','code']

In [6]:
df.drop(columns_to_remove,axis=1,inplace=True)
len(df.columns)

80

In [7]:
df = df[(df.status != "u")]
len(df)

618

In [8]:
train = []
test = []
for i in tqdm(range(len(df['id']))):
    player_data = requests.get('https://fantasy.premierleague.com/api/element-summary/'+str(df['id'].iloc[i])+'/')
    player_data = player_data.json()
    for i in player_data['history']:
        train.append(i)
    for i in player_data['fixtures']:
        test.append(i)

 62%|██████▏   | 384/618 [08:17<05:03,  1.30s/it]


TypeError: string indices must be integers, not 'str'